In [361]:
import warnings, numpy as np, pandas as pd, pymysql, matplotlib, matplotlib.pyplot as plt, seaborn as sb, geopandas as gp, geopy
from datetime import datetime, timedelta

In [362]:
def binner(x):
    h = int(x.strftime('%H'))
    d = int(x.strftime('%w'))
    
    if h >= 4 and h < 15:
        h = 1
    elif h >= 15 and h < 21:
        h = 2
    else:
        h = 3
        
    if d == 0 or d == 6:
        d = 1
    else:
        d = 2
        
    
    
    return "%d%d" % (d,h)

In [363]:
multe = pd.read_csv("/mnt/docs/Progetti/BigDive/Workspace/Final_Project/multe_pos.csv", parse_dates=["date"])
park = pd.read_csv("/mnt/docs/Progetti/BigDive/Workspace/Final_Project/June_2017_correct.csv", parse_dates=["date"])

multe = multe.rename({'num':'num_multe'}, axis=1).drop('via', axis = 1).dropna()
multe['datebin'] = multe['date'].apply(binner)
multe = multe.loc[:, ["datebin", "long", "lat", "num_multe"]].copy()
multe.loc[:,["long", "lat"]] = multe.loc[:,["long", "lat"]].round(4)

park = park.rename({'num':'num_park'}, axis=1)
park['datebin'] = park['date'].apply(binner)
park = park.loc[:, ["datebin", "long", "lat", "num_park"]].copy()
park.loc[:,["long", "lat"]] = park.loc[:,["long", "lat"]].round(4)

""

''

In [364]:
park.head()


,datebin,long,lat,num_park
0,23,7.6035,45.029,1
1,23,7.6035,45.067,1
2,23,7.6035,45.079,2
3,23,7.6205,45.048,1
4,23,7.6205,45.054,1


In [365]:
multe.head()

,datebin,long,lat,num_multe
0,23,7.6487,45.0438,1
1,23,7.6485,45.0439,1
2,23,7.6487,45.0438,1
3,23,7.6487,45.0438,1
4,23,7.6483,45.0439,1


In [366]:
#pd.date_range(np.min(park['date']), np.max(park['date']), periods=12, freq='H')

#create the data bins from the data (already binned in the dataset)
(_, longbins) = pd.cut(park['long'], 20, retbins=True)
(_, latbins) = pd.cut(park['lat'], 20, retbins=True)

park.head()

,datebin,long,lat,num_park
0,23,7.6035,45.029,1
1,23,7.6035,45.067,1
2,23,7.6035,45.079,2
3,23,7.6205,45.048,1
4,23,7.6205,45.054,1


In [367]:
multe['long'] = pd.to_numeric(pd.cut(multe['long'], bins=longbins).apply(lambda x: (x.left+x.right)/2))
multe['lat'] = pd.to_numeric(pd.cut(multe['lat'], bins=latbins).apply(lambda x: (x.left+x.right)/2))

multe.head()

,datebin,long,lat,num_multe
0,23,7.648,45.042,1
1,23,7.648,45.042,1
2,23,7.648,45.042,1
3,23,7.648,45.042,1
4,23,7.648,45.042,1


In [368]:
multe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35327 entries, 0 to 40326
Data columns (total 4 columns):
datebin      35327 non-null object
long         35315 non-null float64
lat          35310 non-null float64
num_multe    35327 non-null int64
dtypes: float64(2), int64(1), object(1)
memory usage: 1.3+ MB


In [369]:
multe.head()

,datebin,long,lat,num_multe
0,23,7.648,45.042,1
1,23,7.648,45.042,1
2,23,7.648,45.042,1
3,23,7.648,45.042,1
4,23,7.648,45.042,1


In [370]:
multe = multe.set_index(["datebin","long","lat"], drop=True)
multe

num_multe
datebin long   lat              
23      7.6480 45.042          1
               45.042          1
               45.042          1
               45.042          1
               45.042          1
               45.042          1
               45.042          1
        7.6885 45.090          1
        7.6480 45.066          1
        7.6970 45.096          1
        7.6400 45.066          1
               45.066          1
               45.066          1
               45.066          1
        7.6480 45.066          1
21      7.6560 45.114          1
        7.6720 45.096          1
               45.096          1
        7.6800 45.096          1
               45.096          1
        7.6320 45.024          1
        7.6560 45.078          1
        7.6480 45.036          1
               45.036          1
               45.036          1
               45.036          1
               45.036          1
               45.042          1
        7.6320 45.078          1
        7.6480 45.042          1
...                          ...
22      7.6640 45.054          1
        7.6720 45.048          1
        7.6800 45.054          1
        7.6640 45.054          1
        7.6800 45.048          1
        7.6640 45.054          1
        7.6720 45.048          1
               45.042          1
               45.042          1
               45.048          1
               45.042          1
               45.042          1
               45.042          1
        7.6800 45.054          1
        7.6720 45.042          1
        7.6800 45.042          2
        7.6720 45.048          1
        7.6800 45.042          1
        7.6640 45.054          1
               45.054          1
        7.6400 45.042          1
        7.6800 45.042          1
        7.6640 45.096          1
        7.6720 45.066          1
        7.6800 45.042          1
        7.6240 45.078          1
        7.6560 45.054          1
               45.054          1
        7.6800 45.066          1
23      7.7130 45.072          1

[35327 rows x 1 columns]

In [371]:
park = park.set_index(["datebin","long","lat"], drop=True)


In [372]:
park['23']

KeyError: '23'

In [ ]:
multe.info()

In [ ]:
joined = multe.join(park, how="outer")

joined = joined.sort_values("num_multe", ascending=False).fillna(0)

joined.index

In [ ]:
sb.pairplot(joined)

In [ ]:
joined